In [250]:
#importing libraries
import requests
import lxml
import lxml_html_clean
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from pprint import pprint
import re
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [251]:
#Webscraping section

In [252]:
#getting url-setup

#This url didn't work because of the 'show more' button making things difficult
#url = "https://movies.disney.com/a-z"
url = "https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films"
headers = {"User-Agent": "For class web scraping assignment, email: cnelson1845@gmail.com"}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text)
print(r.status_code)

200


In [253]:
#using pandas to get all the tables and then turning the relevant ones into dataframes
tables = pd.read_html(r.text)
#getting the relevant tables of past and current movies (no future)
tables = tables[0:9]
dfs = []
for table in tables:
    dfs.append(table)
print(dfs)

C:\Users\cnels\AppData\Local\Temp\ipykernel_16816\159520408.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


[          Release date                                   Title  \
0    December 21, 1937         Snow White and the Seven Dwarfs   
1     February 7, 1940                               Pinocchio   
2    November 13, 1940                                Fantasia   
3        June 20, 1941                    The Reluctant Dragon   
4     October 23, 1941                                   Dumbo   
5      August 21, 1942                                   Bambi   
6     February 6, 1943                          Saludos Amigos   
7        July 17, 1943               Victory Through Air Power   
8     February 3, 1945                    The Three Caballeros   
9       April 20, 1946                         Make Mine Music   
10   November 12, 1946                       Song of the South   
11  September 27, 1947                      Fun and Fancy Free   
12        May 27, 1948                             Melody Time   
13   November 29, 1948                     So Dear to My Heart   
14     Oc

In [254]:
#combining the dataframes
movies = pd.concat(dfs, ignore_index=True)
print(movies)

          Release date                            Title  \
0    December 21, 1937  Snow White and the Seven Dwarfs   
1     February 7, 1940                        Pinocchio   
2    November 13, 1940                         Fantasia   
3        June 20, 1941             The Reluctant Dragon   
4     October 23, 1941                            Dumbo   
..                 ...                              ...   
534     April 22, 2025     Sea Lions of the Galapagos ‡   
535       May 23, 2025                    Lilo & Stitch   
536      June 20, 2025                             Elio   
537     August 8, 2025                  Freakier Friday   
538   October 10, 2025                       Tron: Ares   

                                                 Notes  
0    first film to be distributed by RKO Radio Pict...  
1     Inducted into the National Film Registry in 1994  
2    anthology film Inducted into the National Film...  
3          fictionalized tour around the Disney studio  
4     

In [255]:
#removing the irrelevant column of description
movies = movies.drop('Notes', axis=1)
print(movies.head())

        Release date                            Title
0  December 21, 1937  Snow White and the Seven Dwarfs
1   February 7, 1940                        Pinocchio
2  November 13, 1940                         Fantasia
3      June 20, 1941             The Reluctant Dragon
4   October 23, 1941                            Dumbo


In [256]:
#Getting the title and release year from each column and putting it in a dictionary
the_movies = {}
for idx, movie in movies.iterrows():
    #getting title
    title = movie["Title"]
    #getting year
    year = movie["Release date"][-4:]
    #placing into dictionary
    the_movies[title] = year
print(the_movies)

{'Snow White and the Seven Dwarfs': '1937', 'Pinocchio': '1940', 'Fantasia': '1940', 'The Reluctant Dragon': '1941', 'Dumbo': '2019', 'Bambi': '1942', 'Saludos Amigos': '1943', 'Victory Through Air Power': '1943', 'The Three Caballeros': '1945', 'Make Mine Music': '1946', 'Song of the South': '1946', 'Fun and Fancy Free': '1947', 'Melody Time': '1948', 'So Dear to My Heart': '1948', 'The Adventures of Ichabod and Mr. Toad': '1949', 'Cinderella': '2015', 'Treasure Island': '1950', 'Alice in Wonderland': '2010', 'The Story of Robin Hood': '1952', 'Peter Pan': '1953', 'The Sword and the Rose': '1953', 'The Living Desert': '1953', 'Rob Roy: The Highland Rogue': '1954', 'The Vanishing Prairie': '1954', '20,000 Leagues Under the Sea': '1954', 'Davy Crockett: King of the Wild Frontier': '1955', 'Lady and the Tramp': '1955', 'The African Lion': '1955', 'The Littlest Outlaw': '1955', 'The Great Locomotive Chase': '1956', 'Davy Crockett and the River Pirates': '1956', 'Secrets of Life': '1956', 

In [ ]:
#getting Rotten Tomatoes urls for all the titles
#Note: Rotten Tomatoes does the title all lowercase with no special characters, and all spaces replaced by underscores
#Ex- Terminator: Dark Fate
#url is- https://www.rottentomatoes.com/m/terminator_dark_fate
#format: https://www.rottentomatoes.com/m/{title}
#Notes: If there are multiple version they either have the year after the movie title:
#Ex: https://www.rottentomatoes.com/m/the_little_mermaid_1989
#or they add a little internal number before the title 
#Ex: https://www.rottentomatoes.com/m/1074108-101_dalmatians
#The last one is worst case scenario and we look for it last, for all of these urls we will need to check if we get a proper request back

urls = []
clean_dict = {}
base = "https://www.rottentomatoes.com/m/"
for title in the_movies:
    #formatting title for rotten tomatoes
    clean = title
    clean = clean.replace("[N 1]","")
    #all lowercase
    clean = clean.lower()
    #removing special characters
    clean = clean.replace(":","").replace("!","").replace("/","").replace("'","").replace(",","").replace(".","")
    clean = clean.replace(" ‡","").replace(" †","").replace(" §","").replace(" *","").replace("½","12")
    #replacing some other special characters
    clean = clean.replace("&","and").replace("í","i")
    #replacing spaces with underscores
    clean = clean.replace(" ","_").replace("-","_")
    #adding it to another dictionary to keep track of years
    clean_dict[clean] = the_movies[title]
    #creating full url with it (make sure year is still there)
    full_url = f"{base}{clean}"
    urls.append(full_url)
print(len(urls))
print(urls)

525
['https://www.rottentomatoes.com/m/snow_white_and_the_seven_dwarfs', 'https://www.rottentomatoes.com/m/pinocchio', 'https://www.rottentomatoes.com/m/fantasia', 'https://www.rottentomatoes.com/m/the_reluctant_dragon', 'https://www.rottentomatoes.com/m/dumbo', 'https://www.rottentomatoes.com/m/bambi', 'https://www.rottentomatoes.com/m/saludos_amigos', 'https://www.rottentomatoes.com/m/victory_through_air_power', 'https://www.rottentomatoes.com/m/the_three_caballeros', 'https://www.rottentomatoes.com/m/make_mine_music', 'https://www.rottentomatoes.com/m/song_of_the_south', 'https://www.rottentomatoes.com/m/fun_and_fancy_free', 'https://www.rottentomatoes.com/m/melody_time', 'https://www.rottentomatoes.com/m/so_dear_to_my_heart', 'https://www.rottentomatoes.com/m/the_adventures_of_ichabod_and_mr_toad', 'https://www.rottentomatoes.com/m/cinderella', 'https://www.rottentomatoes.com/m/treasure_island', 'https://www.rottentomatoes.com/m/alice_in_wonderland', 'https://www.rottentomatoes.com

In [258]:
#Note: This box takes awhile, I'm talking about 10-15 minutes

#making a dictionary to mark the bad ones
marked = {}
#checking all the urls, if they fail, we add the year
for i, url in enumerate(urls):
    #create a request object and check if the status code is 200 or not
    r = requests.get(url)
    if not r.status_code == 200:
        #trying to fix it by removing 'the'
        current = url.replace("the_","",1)
        s = requests.get(current)
        if s.status_code == 200:
            urls[i] = current
            continue
        #trying to fix by removing a
        current = url.replace("a_","",1)
        s = requests.get(current)
        if s.status_code == 200:
            urls[i] = current
            continue
        #we try fixing it by adding the year
        title = url.rsplit("/",1)[1]
        current = url+f"_{clean_dict[title]}"
        s = requests.get(current)
        if s.status_code == 200:
            urls[i] = current
            continue
        #we try and fix it by changing roman numeral ii to number 2
        current = url.replace("ii","2")
        s = requests.get(current)
        if s.status_code == 200:
            urls[i] = current
            continue
        #adding to marked since it couldn't be easily fixed
        marked[i] = url
#pretty printing it so I can read it
pprint(marked)

{7: 'https://www.rottentomatoes.com/m/victory_through_air_power',
 18: 'https://www.rottentomatoes.com/m/the_story_of_robin_hood',
 24: 'https://www.rottentomatoes.com/m/20000_leagues_under_the_sea',
 36: 'https://www.rottentomatoes.com/m/the_light_in_the_forest',
 37: 'https://www.rottentomatoes.com/m/white_wilderness',
 43: 'https://www.rottentomatoes.com/m/toby_tyler_or_10_weeks_with_a_circus',
 44: 'https://www.rottentomatoes.com/m/kidnapped',
 51: 'https://www.rottentomatoes.com/m/the_absent_minded_professor',
 58: 'https://www.rottentomatoes.com/m/big_red',
 71: 'https://www.rottentomatoes.com/m/the_moon_spinners',
 83: 'https://www.rottentomatoes.com/m/the_gnome_mobile',
 192: 'https://www.rottentomatoes.com/m/angels_in_the_outfield',
 195: 'https://www.rottentomatoes.com/m/rudyard_kiplings_the_jungle_book',
 218: 'https://www.rottentomatoes.com/m/george_of_the_jungle',
 227: 'https://www.rottentomatoes.com/m/serengeti_symphony',
 230: 'https://www.rottentomatoes.com/m/mighty_jo

In [ ]:
#manually fixing the bad urls
urls[7] = "https://www.rottentomatoes.com/m/victory-through-air-power"
urls[18] = "https://www.rottentomatoes.com/m/story_of_robin_hood_and_his_merrie_men"
urls[24] = "https://www.rottentomatoes.com/m/1000079-20000_leagues_under_the_sea"
urls[36] = "https://www.rottentomatoes.com/m/1012343-light_in_the_forest"
urls[37] = "https://www.rottentomatoes.com/m/truelife_adventures_white_wilderness"
urls[43] = "https://www.rottentomatoes.com/m/toby_tyler"
urls[44] = "https://www.rottentomatoes.com/m/1011511-kidnapped"
urls[51] = "https://www.rottentomatoes.com/m/absentminded_professor"
urls[58] = "https://www.rottentomatoes.com/m/1002343-big_red"
urls[71] = "https://www.rottentomatoes.com/m/moonspinners"
urls[83] = "https://www.rottentomatoes.com/m/gnomemobile"
urls[192] = "https://www.rottentomatoes.com/m/1055833-angels_in_the_outfield"
urls[195] = "https://www.rottentomatoes.com/m/1057637-jungle_book"
urls[218] = "https://www.rottentomatoes.com/m/1077847-george_of_the_jungle"
urls[227] = "" #There is no entry in rotten tomatoes for Serengeti Symphony, will delete at end
urls[230] = "https://www.rottentomatoes.com/m/1084582-mighty_joe_young"
urls[262] = "https://www.rottentomatoes.com/m/spirited_away" #different
urls[265] = "https://www.rottentomatoes.com/m/winnie-the-pooh-a-very-merry-pooh-year"
urls[285] = "https://www.rottentomatoes.com/m/winnie_the_pooh_springtime_with_roo"
urls[287] = "https://www.rottentomatoes.com/m/1141548-sacred_planet"
urls[291] = "https://www.rottentomatoes.com/m/the-three-musketeers2004"
urls[302] = "https://www.rottentomatoes.com/m/howls_moving_castle" #different
urls[305] = "https://www.rottentomatoes.com/m/lilo_and_stitch_2"
urls[309] = "https://www.rottentomatoes.com/m/chronicles_of_narnia_lion_witch_wardrobe"
urls[311] = "https://www.rottentomatoes.com/m/1155467-glory_road"
urls[321] = "https://www.rottentomatoes.com/m/the_nightmare_before_christmas"
urls[322] = "https://www.rottentomatoes.com/m/santa_clause_3"
urls[332] = "https://www.rottentomatoes.com/m/the_gameplan"
urls[342] = "https://www.rottentomatoes.com/m/10008587-beverly_hills_chihuahua"
urls[344] = "https://www.rottentomatoes.com/m/1194524-high_school_musical_3_senior_year"
urls[350] = "https://www.rottentomatoes.com/m/jonas_brothers_3d"
urls[354] = "https://www.rottentomatoes.com/m/trail-of-the-panda"
urls[357] = "https://www.rottentomatoes.com/m/hexe_lilli_der_drache_und_das_magische_buch"
urls[358] = "https://www.rottentomatoes.com/m/10009462-g_force"
urls[360] = "https://www.rottentomatoes.com/m/walt_and_el_grupo" #different
urls[361] = "" #there is no entry for SpangaS op Survival, thus it will be deleted at the end
urls[364] = "https://www.rottentomatoes.com/m/10008502-christmas_carol"
urls[366] = "https://www.rottentomatoes.com/m/10009596-old_dogs"
urls[368] = "https://www.rottentomatoes.com/m/high_school_musical_el_desafio"
urls[374] = "https://www.rottentomatoes.com/m/gedo-senki-tales-from-earthsea"
urls[382] = "https://www.rottentomatoes.com/m/10011582-tron_legacy"
urls[433] = "https://www.rottentomatoes.com/m/tini_el_gran_cambio_de_violetta"
urls[441] = "https://www.rottentomatoes.com/m/march_of_the_penguins_2_the_next_step"
urls[449] = "https://www.rottentomatoes.com/m/disneynature_expedition_china"
urls[485] = "https://www.rottentomatoes.com/m/the_last_warrior_2017"
urls[487] = "https://www.rottentomatoes.com/tv/the_beatles_get_back"
#Note: The null entries were Dutch in origin

#deleted null entries
urls.pop(361)
urls.pop(227)

''

In [260]:
#getting relevant information from each url
title = []
audience_score = []
critic_score = []
revenue = []
genre = []
awards = []
#This is film rating-PG,R,etc
rating = []
#We will get the full date now, but we will likely need to split it into month and year (and season) later
release_date = []
for url in urls:
    #getting title

    #getting audience score

    #getting critic score

    #getting rating

    #getting box-office revenue

    #getting genre

    #getting release date

    #getting awards (may be none)

    pass



In [261]:
#putting the information into a dataframe
df = pd.DataFrame({"Title":title,"Audience-Score":audience_score,"Critic-Score":critic_score,"Revenue":revenue,"Release-Date":release_date,"Film-Rating":rating,"Genre":genre,"Awards":awards})
#printing to check what we will need to clean next
print(df.head())

Empty DataFrame
Columns: [Title, Audience-Score, Critic-Score, Revenue, Release-Date, Film-Rating, Genre, Awards]
Index: []


In [262]:
#cleaning the dataframe
#Note for me:
'''
df["Team"] = df.apply(lambda row: (re.sub(r"\[[A-Z]*\]","",row.Team)), axis=1)
df["Team"] = df.apply(lambda row: (re.sub(r"\([A-Z]*\)","",row.Team)), axis=1)
df['Team'] = df['Team'].replace(r'\s+$', '', regex=True)
df
'''

<>:4: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\['
C:\Users\cnels\AppData\Local\Temp\ipykernel_16816\2630949362.py:4: SyntaxWarning: invalid escape sequence '\['
  df["Team"] = df.apply(lambda row: (re.sub(r"\[[A-Z]*\]","",row.Team)), axis=1)


'\ndf["Team"] = df.apply(lambda row: (re.sub(r"\\[[A-Z]*\\]","",row.Team)), axis=1)\ndf["Team"] = df.apply(lambda row: (re.sub(r"\\([A-Z]*\\)","",row.Team)), axis=1)\ndf[\'Team\'] = df[\'Team\'].replace(r\'\\s+$\', \'\', regex=True)\ndf\n'

In [ ]:
#creating month, year, decade, and season column from "Release-Date" and deleting release date column
#decade creation
df["Decade"] = (df["Release-Date"] // 10) * 10
#sseson creation
df['season'] = df[]
#



In [ ]:
#saving the dataframe as a csv



In [ ]:
#Analysis Section - From here on, rating refers to score unless otherwise specified

In [ ]:
#What is the average movie rating each year? (audience and critic)

Aud_mean_year = df.groupby('Release-Date')['Audience-Score'].mean()
Crit_mean_year = df.groupby('Release-Date')['Critic-Score'].mean()
Ave_mean_year = (Aud_mean_year + Crit_mean_year)/2

print(f'Audience score by year' {Aud_mean_year})
print(f'Critic score by year' {Crit_mean_year})
print(f'The avrage of the two scores' {Ave_mean_year})

In [ ]:
#Average movie rating each decade (audience and critic)

df["Decade"] = (df["Release-Date"] // 10) * 10


Aud_mean_dec = df.groupby('Decade')['Audience-Score'].mean()
Crit_mean_dec = df.groupby('Decade')['Critic-Score'].mean()
ave_mean_dec = ((Aud_mean_dec+Crit_mean_dec)/2).mean()

In [268]:
#Note section:
#What is the trend in ratings? (rising, falling, stagnant, other)

#Is it different between audiences and critics? If so, by what degree? 

#Any other anomalies you notice? Any dips or highs? Why might that be?


In [ ]:
#What is the average box office revenue each year? 

Revenue_mean_year = df.groupby('year')['Revenue'].mean()


In [ ]:
#Revenue for each decade
Revenue_mean_year = df.groupby('Decade')['Revenue'].mean()

In [ ]:
#Revenue for each season? 


In [ ]:
#Does revenue correlate with audience ratings?

sns.scatterplot(data = df, x = 'Revenue', y = 'Audience-Score')


In [ ]:
#Does revenue correlate with critic ratings?

sns.scatterplot(data = df, x = 'Revenue', y = 'Critic-Score')

In [ ]:
#Does revenue correlate more strongly with audience or critic ratings?



In [274]:
#Note: What is the trend regarding revenue?

#Please note any other anomalies and your explanation (it's okay if you don't know)


In [275]:
#How many total directors have there been? 


In [276]:
#How many different directors have there been each year? Each decade?


In [277]:
#What is the average revenue of movies made by each director?


In [278]:
#What is the average rating of movies made by each director?


In [279]:
#Do directors with more movies have better ratings?


In [280]:
#Do directors movies get better ratings as they make more movies?


In [281]:
#Note: Make any notes about directors and relationships with other variables



In [282]:
#Any interesting relationships between film rating and rating?


In [283]:
#What about film rating and revenue? 


In [284]:
#Note: Note anything interesting regarding film rating



In [285]:
#Any interesting relationships between genre and rating?


In [286]:
#What about genre and revenue?


In [287]:
#What is the count of different film ratings in each genre?


In [288]:
#Note: Add anything interesting regarding genre



In [289]:
#What is the average number of awards a movie gets?


In [290]:
#What is the average number of movie awards for each decade? Does it show a trend?


In [291]:
#What about year? Does it show any trends?


In [292]:
#Note: Add anything interesting regarding awards


In [293]:
#Note: Add any overall thoughts

